In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import add_funcs as af
import os

from scipy import integrate
from astropy.io import ascii
from tqdm import tqdm

In [ ]:
#Set some global parameters

lambda_min = 5000
lambda_max = 8000

norm = 'OI' #Either 'OI' (normalising by the integrated flux of a fitted OI gaussian)
            #or 'Total' (normalising by integrated flux over range lambda_min to lambda_max)
            #or 'AJ' (normalising by Lnorm from AJ 2015)

In [ ]:
#Load in the models:

string_epochs = np.array(['200d', '250d', '300d', '350d', '400d']) #Days
string_mixings = np.array(['10_90', '20_80', '30_70', '60_40'])
string_masses = np.array(['3p30', '4p00', '6p00'])

wl_lists, flux_lists, NII_flux_lists = [], [], []

for i in range(len(string_epochs)):
    
    epoch = string_epochs[i]
    local_wl_list, local_flux_list, local_NII_flux_list = [], [], []
    
    for k in range(len(string_masses)):
        for l in range(len(string_mixings)):
            
            model_name = '/NII_he' + string_masses[k] + '_AJ_' + string_mixings[l]
            spectrum_path = "/home/stba7609/Documents/Projects/" + epoch + model_name + "/out/modelspectra/spectrum.datrun001"
            data = ascii.read(spectrum_path)
            
            local_wl_list.append(data['col1'])
            local_flux_list.append(data['col2'])
            local_NII_flux_list.append(data['col15'])
            
    wl_lists.append(local_wl_list)
    flux_lists.append(local_flux_list)
    NII_flux_lists.append(local_NII_flux_list)

In [ ]:
#Get the NII fitted fluxes
M_sun = 1.989*10**33 #g

NII_percentages_fit = np.zeros((3, 4, 5)) #3 masses, 4 mixings, 5 epochs
ni_masses = np.array([0.04, 0.0445, 0.0704])*M_sun
epochs = np.array([200, 250, 300, 350, 400])

distance = 10*10**6 *3.086*10**18 #10 Mpc
surface = 4*np.pi*distance**2

for i in tqdm(range(len(string_masses))):
    M_Ni = ni_masses[i]
    for j in tqdm(range(len(string_mixings))):
        for k in range(len(string_epochs)):
            
            wl, flux = np.array(wl_lists[k][4*i + j]), np.array(flux_lists[k][4*i + j])
            
            zoom_mask = (wl > lambda_min) * (wl < lambda_max)
            wl_zoom, flux_zoom = wl[zoom_mask], flux[zoom_mask]
            
            NII_integrated_flux, popt = af.observed_flux_from_fit(wl_zoom, flux_zoom)
            
            if norm == 'OI':
                norm_mask2 = (wl > 6100) * (wl < 6800)
                OI_flux = af.gaussian(wl[norm_mask2], 6316, popt[0], popt[2])
                normalisation_flux = integrate.cumtrapz(OI_flux, wl[norm_mask2])[-1]
                print(NII_integrated_flux*100/normalisation_flux, i, j)
            elif norm == 'Total':
                normalisation_flux = integrate.cumtrapz(flux_zoom, wl_zoom)[-1]
            elif norm == 'AJ':
                epoch = epochs[k]
                normalisation_flux = af.Lnorm_AJ(epoch, M_Ni)/surface
            
            NII_percentages_fit[i, j, k] = NII_integrated_flux*100/normalisation_flux
    

In [ ]:
plt.rcParams["font.size"] = 20

In [ ]:
print(NII_percentages_fit)
print(af.Lnorm_AJ(200, 0.04*M_sun))

In [ ]:
#Make the plot
fig, ax = plt.subplots(1, 3, sharex = True, sharey = True)

colours = ['green', 'yellow', 'orange', 'red']

epochs_2011dh = np.array([152, 202, 208, 239, 268, 292, 301, 359, 416])
percentages_2011dh_4200 = np.array([3.5, 3.7, 3.7, 5.1, 5.4, 6.4, 6.7, 7.8, 8.4 ])
percentages_2011dh_5000 = np.array([4.4, 4.9, 5.2, 5.8, 7.3, 8.2, 8.5, 11, 11])
percentages_2011dh_AJnorm = np.array([0.533, 0.364, 0.379, 0.303, 0.31, 0.3, 0.256, 0.259, 0.179])

epochs_2007Y = np.array([219, 248, 288])
percentages_2007Y_4200 = np.array([5.2, 7.9, 9.2])
percentages_2007Y_5000 = np.array([6.8, 10.1, 11.6])
percentages_2007Y_AJnorm = np.array([0.196, 0.193, 0.139])

epochs_2008ax = np.array([280, 328, 359])
percentages_2008ax_5000 = np.array([5.2, 5.3, 9])

epochs_2004dk = np.array([283, 342])
percentages_2004dk_5000 = np.array([2.54, 3.1])

epochs = np.array([200, 250, 300, 350, 400])

for i in range(len(string_masses)):
    
    
    for j in range(1):
        
        this_label = "Fit he" + string_masses[i] + "_" + string_mixings[j]
        ax[i].plot(epochs, NII_percentages_fit[i, j, :], color = colours[j], label = this_label)
        
    #ax[i].plot(epochs_2011dh, percentages_2011dh_4200, linestyle = '--', color = 'black', label = '2011dh')
    #ax[i].plot(epochs_2011dh, percentages_2011dh_5000, linestyle = '--', color = 'black', label = '2011dh')
    #ax[i].plot(epochs_2007Y, percentages_2007Y_5000, linestyle = ':', color = 'black', label = '2007Y')
    #ax[i].plot(epochs_2008ax, percentages_2008ax_5000, linestyle = '-', color = 'black', label = '2008ax')
    #ax[i].plot(epochs_2004dk, percentages_2004dk_5000, linestyle = '-.', color = 'black', label = '2004dk')
    ax[i].set_xlabel('Time since explosion [d]')
    ax[i].legend(fontsize = 14)
    
ax[0].set_ylabel('Percentage flux in the NII doublet')
fig.set_size_inches(20, 12)

#plt.savefig('NII_Time_Tracks_newff_Totalnorm.png')
plt.show()

In [ ]:
for i in range(5):
    
    wl, flux = np.array(wl_lists[i][0 + 3]), np.array(flux_lists[i][0 + 3])
            
    zoom_mask = (wl > lambda_min) * (wl < lambda_max)
    wl_zoom, flux_zoom = wl[zoom_mask], flux[zoom_mask]
    
    plt.plot(wl_zoom, flux_zoom/np.max(flux_zoom))
    
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()